# BLP method

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm

In [2]:
# read data
data = pd.read_csv('AggDemand.csv', header=None)

In [3]:
data.head()

,0,1,2,3,4,5
0,6488,90808,2020,78,0,1
1,8748,228211,2390,102,0,1
2,11588,284595,2690,115,0,1
3,12268,78521,2500,103,0,1
4,14898,14257,2599,130,0,1


In [4]:
p = np.log(np.asarray(data[0])) # log of prices

In [5]:
q = np.asarray(data[1]) # log of ???

In [6]:
num = p.shape[0] # number of individuals

In [7]:
# independent variables
x = np.stack([np.ones(num),
                        np.log(np.asarray(data[2])),
                        np.log(np.asarray(data[3])),
                        np.asarray(data[4])],
                       axis=1)

In [8]:
# market size
MS = 50000000

# number of simulations
ns = 1000

# market share for outside option
s0 = (MS - sum(q))/MS

# market share of each product
sj = q / MS

In [9]:
y = np.log(sj) - np.log(s0)

In [10]:
# estimated coefficients
beta = np.dot(np.dot(np.linalg.inv(np.dot(x.T, x)),  x.T) ,y)
beta

array([-24.25106987,   3.28942105,  -1.77117311,  -0.91423405])

In [11]:
# random numbers
nu = norm() .rvs((ns, 2)) # standard normal distribution

In [12]:
# initial delta
init_delta = np.log(sj) - np.log(s0)
sigma = np.array([0.5, 0.5])

In [13]:
deltas = np.empty((num, ns))
for n in range(num):
    for k in range(ns):
        deltas[n,k] = np.dot(beta, x[n, :])

mus = np.empty((num, ns))
for row in range(num):
    for col in range(ns):
        y = [x[row, k+1] * sigma[k] * nu[col,k] for k in range(2)]
        mus[row, col] = sum(y)
        
exp_mat = np.exp(deltas + mus)

mat = np.empty((num, ns))
for col in range(ns):
    denominator = 1 + sum(exp_mat[:, col])
    for row in range(num):
        mat[row, col] = exp_mat[row, col] / denominator

In [14]:
pd.DataFrame(mat)

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0.000150,0.001431,0.000100,0.000240,2.613521e-06,0.000367,0.006157,2.203680e-06,3.111181e-06,0.005562,...,0.004683,2.981732e-06,0.000232,0.000504,0.003060,0.002898,0.005296,0.000107,0.003899,0.005249
1,0.000151,0.001361,0.000107,0.000252,2.644938e-06,0.000389,0.007194,1.971360e-06,2.522488e-06,0.007036,...,0.005745,2.679630e-06,0.000214,0.000551,0.003227,0.003022,0.006029,0.000101,0.004493,0.007031
2,0.000173,0.001569,0.000124,0.000294,2.935267e-06,0.000457,0.009178,2.104080e-06,2.620987e-06,0.009175,...,0.007308,2.871668e-06,0.000240,0.000656,0.003877,0.003616,0.007513,0.000113,0.005573,0.009409
3,0.000170,0.001576,0.000120,0.000285,2.893159e-06,0.000440,0.008466,2.178295e-06,2.820640e-06,0.008203,...,0.006624,2.966793e-06,0.000244,0.000624,0.003735,0.003499,0.007012,0.000114,0.005181,0.008216
4,0.000121,0.001000,0.000093,0.000213,2.285902e-06,0.000329,0.006283,1.485375e-06,1.692309e-06,0.006749,...,0.005454,2.020960e-06,0.000156,0.000483,0.002629,0.002435,0.005242,0.000076,0.004013,0.007100
5,0.000063,0.000531,0.000049,0.000114,1.116087e-06,0.000178,0.003906,6.878725e-07,7.574715e-07,0.004314,...,0.003343,9.424409e-07,0.000079,0.000266,0.001483,0.001365,0.003130,0.000038,0.002369,0.004721
6,0.000055,0.000428,0.000045,0.000102,1.011470e-06,0.000160,0.003626,5.594634e-07,5.625542e-07,0.004309,...,0.003304,7.674175e-07,0.000063,0.000246,0.001297,0.001183,0.002888,0.000031,0.002230,0.004917
7,0.000051,0.000394,0.000043,0.000097,9.709843e-07,0.000153,0.003462,5.208623e-07,5.101533e-07,0.004203,...,0.003219,7.144663e-07,0.000058,0.000236,0.001222,0.001112,0.002758,0.000029,0.002144,0.004847
8,0.000043,0.000303,0.000039,0.000085,8.586334e-07,0.000133,0.003118,4.067372e-07,3.587194e-07,0.004119,...,0.003121,5.585370e-07,0.000044,0.000212,0.001030,0.000928,0.002470,0.000023,0.001966,0.004981
9,0.000172,0.001635,0.000117,0.000280,2.895352e-06,0.000432,0.007923,2.322041e-06,3.162952e-06,0.007368,...,0.006038,3.155731e-06,0.000257,0.000602,0.003677,0.003463,0.006643,0.000119,0.004875,0.007171


In [15]:
compute_delta = np.asarray([1 / 1000 * sum(mat[row,:]) for row in range(128)])
update_deltas = init_delta + np.log(sj) - np.log(compute_delta)
distance = sum([(update_deltas[i] - init_delta[i])**2 for i in range(128)])

# contraction mapping
while distance < 1e-6:
    init_delta[:] = update_deltas[:]
    
    for i in range(1000):
        deltas[:,i] = init_delta[:]
    exp_mat = np.exp(deltas + mus)
    mat = np.empty((num, ns))
    for col in range(ns):
        denominator = 1 + sum(exp_mat[:, col])
        for row in range(num):
            mat[row, col] = exp_mat[row, col] / denominator
            
    compute_delta = np.asarray([1 / 1000 * sum(mat[row,:]) for row in range(1000)])
    update_deltas = init_delta + np.log(sj) - np.log(compute_delta)
    distance = sum([(update_deltas[i] - init_delta[i])**2 for i in range(128)])

print(update_deltas)

[ -6.20130719  -4.56244202  -4.37581946  -6.84836653 -10.0495719
  -8.68716372  -9.79675732 -11.16259483  -7.43690561  -7.67858723
  -6.8572918  -12.92444713  -7.25930823  -5.56628483 -11.27107257
  -8.27058369  -9.64033881  -5.11881438  -6.44000305  -3.54968837
  -8.37692451  -6.87570735  -9.25128865  -8.85153482  -7.30851896
  -7.72502763  -9.62994945  -9.60835321  -7.78476788 -15.1132457
  -8.12459144  -7.46568605 -10.8780335   -9.18896879  -9.49916001
  -7.13892222 -10.18912242 -11.35829283 -11.78657142 -12.27601092
 -10.71525738 -13.72699393  -7.63683571  -8.66685896 -10.5093871
  -9.33670345  -9.1601065  -11.52221011 -12.13580267 -12.72071252
  -7.67330015  -7.96753681  -8.40424698 -11.58901318 -12.20565383
 -12.58352855 -11.50009177  -8.10810967  -8.87847137  -7.53491849
 -15.52152757  -8.97313176  -6.61433564  -7.01562318  -8.40006738
  -8.00970169  -6.93648073  -8.78330739 -10.68251322  -9.91135624
  -8.60847001  -8.0206127  -10.20899749  -7.17977528  -4.04937118
  -8.44202795